# 오버워치 2 점수판 분석 AI

In [1]:
import pandas as pd
from os import path
import joblib

players = 10


def prepare_scoreboards(defeat_, victory_):
    scoreboards_ = pd.concat([defeat_, victory_])
    scoreboards_.set_index(
        pd.Index((i, j) for i in range(len(scoreboards_.index) // players) for j in range(players)), inplace=True
    )
    return scoreboards_


datapath = path.join('datasets', '')
range_ = range(3, 10)
defeat = pd.read_csv(datapath + 'defeat.csv', usecols=range_)
victory = pd.read_csv(datapath + 'victory.csv', usecols=range_)
scoreboards = prepare_scoreboards(defeat, victory)
scoreboards.loc[
    (
            scoreboards['E'] == 0
    ) & (
            scoreboards['A'] == 0
    ) & (
            scoreboards['D'] == 0
    ) & (
            scoreboards['DMG'] == 0
    ) & (
            scoreboards['H'] == 0
    ) & (
            scoreboards['MIT'] == 0
    ), 'D'
] = scoreboards['D'].max()
scoreboards

E   A   D    DMG      H   MIT    GAME
0   0   7   3   6   5933   1068  6199  DEFEAT
    1   7   0  10   3601    747     0  DEFEAT
    2   9   1   6   4352      0     0  DEFEAT
    3   7   7   2   2332   6325     0  DEFEAT
    4   6   4   2   3410   5979   606  DEFEAT
...    ..  ..  ..    ...    ...   ...     ...
396 5  15   1  13   8895      0  7164  DEFEAT
    6  14   1  10   9021    264     0  DEFEAT
    7  17   2   8  13526      0   317  DEFEAT
    8   1  15   8   1703  10138   178  DEFEAT
    9   5  14   4   2553  14656     0  DEFEAT

[3970 rows x 7 columns]

In [2]:
from sklearn import utils

count = len(scoreboards.index) // players
scoreboards = scoreboards.loc[utils.shuffle(range(count))]
scoreboards

E   A   D    DMG      H    MIT     GAME
186 0  19   3  11  12744      0  22464   DEFEAT
    1  18   1   5   9596      0      0   DEFEAT
    2  14   2  17   8583   1116      0   DEFEAT
    3  13   6   9   6356  10715    319   DEFEAT
    4   6   5  12   3072   9568     24   DEFEAT
...    ..  ..  ..    ...    ...    ...      ...
99  5   9   1   3   5547      0   9096  VICTORY
    6  13   0   3   4340   1288      0  VICTORY
    7   8   0   1   4513      0    106  VICTORY
    8   0  12   2      0   3520      0  VICTORY
    9  10   5   2   2532   4761      0  VICTORY

[3970 rows x 7 columns]

In [3]:
X = scoreboards.drop('GAME', axis=1)
y = scoreboards['GAME'][::players].copy()

In [4]:
for i in range(count):
    X.loc[i, 0] -= X.loc[i, 5]
    X.loc[i, 1] += X.loc[i, 2] - X.loc[i, 6] - X.loc[i, 7]
    X.loc[i, 3] += X.loc[i, 4] - X.loc[i, 8] - X.loc[i, 9]
    X.drop([(i, 2), (i, 4), (i, 5), (i, 6), (i, 7), (i, 8), (i, 9)], inplace=True)

In [5]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler().fit(X)
X = pd.DataFrame(scaler.transform(X), X.index, X.columns)
X

E         A         D       DMG         H       MIT
186 0  0.284483  0.346939  0.619718  0.411329  0.132187  0.656081
    1  0.215517  0.438776  0.577465  0.240778  0.128061  0.477599
    3  0.482759  0.142857  0.661972  0.585619  0.098904  0.479397
4   0  0.474138  0.459184  0.549296  0.443868  0.154490  0.322256
    1  0.344828  0.408163  0.633803  0.367423  0.123025  0.463637
...         ...       ...       ...       ...       ...       ...
268 1  0.327586  0.469388  0.521127  0.230250  0.129375  0.428622
    3  0.448276  0.326531  0.718310  0.451650  0.141600  0.504877
99  0  0.431034  0.479592  0.549296  0.459737  0.132187  0.488015
    1  0.353448  0.459184  0.605634  0.398779  0.123179  0.501501
    3  0.370690  0.387755  0.591549  0.361053  0.139397  0.493952

[1191 rows x 6 columns]

In [6]:
players, features = X.loc[0].shape
total_features = players * features
X = X.values.reshape(-1, total_features)
print(X.shape)
print(y.shape)

(397, 18)
(397,)


In [7]:
from sklearn import ensemble, linear_model, neighbors, neural_network, svm, tree, model_selection
import operator

estimators = (
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(n_jobs=-1),
    ensemble.ExtraTreesClassifier(n_jobs=-1),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(n_jobs=-1),
    ensemble.HistGradientBoostingClassifier(),
    linear_model.LogisticRegression(n_jobs=-1),
    linear_model.LogisticRegressionCV(max_iter=800, n_jobs=-1),
    linear_model.PassiveAggressiveClassifier(n_jobs=-1),
    linear_model.Perceptron(n_jobs=-1),
    linear_model.RidgeClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(n_jobs=-1),
    neighbors.KNeighborsClassifier(n_jobs=-1),
    neighbors.RadiusNeighborsClassifier(2., n_jobs=-1),
    neighbors.NearestCentroid(),
    neural_network.MLPClassifier(max_iter=1600),
    svm.LinearSVC(dual='auto'),
    svm.NuSVC(nu=.0625),
    svm.SVC(),
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier()
)
scores = [(estimator, model_selection.cross_val_score(estimator, X, y, n_jobs=-1).mean()) for estimator in estimators]
scores.sort(key=operator.itemgetter(1), reverse=True)
scores

[(LogisticRegressionCV(max_iter=800, n_jobs=-1), 0.9091772151898734),
 (ExtraTreesClassifier(n_jobs=-1), 0.9042405063291138),
 (LogisticRegression(n_jobs=-1), 0.9042088607594936),
 (NearestCentroid(), 0.9041772151898734),
 (SVC(), 0.8991772151898733),
 (BaggingClassifier(n_jobs=-1), 0.8941455696202532),
 (MLPClassifier(max_iter=1600), 0.8940822784810127),
 (LinearSVC(dual='auto'), 0.8940822784810125),
 (RandomForestClassifier(n_jobs=-1), 0.8916139240506329),
 (RidgeClassifier(), 0.8915506329113925),
 (RidgeClassifierCV(), 0.8865506329113924),
 (KNeighborsClassifier(n_jobs=-1), 0.8815506329113925),
 (GradientBoostingClassifier(), 0.8790822784810126),
 (HistGradientBoostingClassifier(), 0.8740189873417721),
 (SGDClassifier(n_jobs=-1), 0.8637025316455695),
 (AdaBoostClassifier(), 0.8589240506329114),
 (NuSVC(nu=0.0625), 0.84373417721519),
 (ExtraTreeClassifier(), 0.8362974683544303),
 (DecisionTreeClassifier(), 0.8362025316455697),
 (Perceptron(n_jobs=-1), 0.8189873417721518),
 (PassiveAg

In [8]:
defeats = len(defeat) // players


def find_file(index):
    index += 1
    if index <= defeats:
        return f'DEFEAT ({index})'
    else:
        index -= len(victory) // players
        return f'VICTORY ({index})'

In [9]:
for estimator, _ in scores:
    estimator = estimator.fit(X, y)
    if hasattr(estimator, 'feature_importances_'):
        print(estimator)
        importances = estimator.feature_importances_.reshape(-1, features)

        titles = '처치', '도움', '죽음', '피해', '치유', '경감'

        print('\n**특성 중요도**')
        for importance, title in sorted(zip(importances.sum(0), titles), reverse=True):
            print(f'{title}: {importance * 100:.0f}%')

        print('\n**1인 돌격 특성 중요도**')
        for importance, title in sorted(zip(importances[0], titles), reverse=True):
            print(f'{title}: {importance * 100:.0f}%')

        print('\n**1인 공격 특성 중요도**')
        for importance, title in sorted(zip(importances[1] / 2, titles), reverse=True):
            print(f'{title}: {importance * 100:.0f}%')

        print('\n**1인 지원 특성 중요도**')
        for importance, title in sorted(zip(importances[2] / 2, titles), reverse=True):
            print(f'{title}: {importance * 100:.0f}%')

        print('\n**1인 역할 중요도**')
        roles = importances.sum(1)
        for importance, title in sorted(zip((roles[0], roles[1] / 2, roles[2] / 2), ('돌격', '공격', '지원')), reverse=True):
            print(f'{title}: {importance * 100:.0f}%')

        break

ExtraTreesClassifier(n_jobs=-1)

**특성 중요도**
처치: 34%
죽음: 27%
도움: 18%
피해: 10%
경감: 6%
치유: 5%

**1인 돌격 특성 중요도**
처치: 14%
죽음: 9%
도움: 5%
피해: 4%
경감: 2%
치유: 2%

**1인 공격 특성 중요도**
처치: 7%
죽음: 5%
피해: 2%
경감: 1%
도움: 1%
치유: 1%

**1인 지원 특성 중요도**
도움: 5%
죽음: 4%
처치: 3%
피해: 1%
경감: 1%
치유: 1%

**1인 역할 중요도**
돌격: 34%
공격: 17%
지원: 16%


In [10]:
from tensorflow.keras import backend, layers
from tensorflow import keras

backend.clear_session()
model = keras.Sequential()
model.add(layers.Dense(300, 'relu', input_shape=[total_features]))
model.add(layers.Dense(100, 'relu'))
model.add(layers.Dense(3, 'softmax'))
model.summary()

2023-11-06 11:49:57.675090: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               5700      
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 36103 (141.03 KB)
Trainable params: 36103 (141.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-11-06 11:49:58.882981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-06 11:49:58.901754: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-06 11:49:58.902009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-06 11:49:58.903299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-06 11:49:58.903588: I tensorflow/compile

In [11]:
from tensorflow.keras import optimizers, callbacks

encoder = preprocessing.LabelEncoder()
model.compile('sgd', 'sparse_categorical_crossentropy', ['accuracy'])
history = model.fit(
    X,
    encoder.fit_transform(y),
    epochs=100,
    verbose=2,
    callbacks=[callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
    validation_split=.1
)
history.history['val_loss'][-11], history.history['val_accuracy'][-11]

Epoch 1/100


2023-11-06 11:50:00.013356: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-06 11:50:00.027820: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7847b00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-06 11:50:00.027855: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2023-11-06 11:50:00.047021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-11-06 11:50:00.056250: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-06 11:50:00.093230: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/12 - 1s - loss: 1.0984 - accuracy: 0.4846 - val_loss: 1.0143 - val_accuracy: 0.4750 - 827ms/epoch - 69ms/step
Epoch 2/100
12/12 - 0s - loss: 0.9825 - accuracy: 0.4846 - val_loss: 0.9361 - val_accuracy: 0.4750 - 55ms/epoch - 5ms/step
Epoch 3/100
12/12 - 0s - loss: 0.9134 - accuracy: 0.4846 - val_loss: 0.8825 - val_accuracy: 0.4750 - 52ms/epoch - 4ms/step
Epoch 4/100
12/12 - 0s - loss: 0.8649 - accuracy: 0.4846 - val_loss: 0.8417 - val_accuracy: 0.4750 - 56ms/epoch - 5ms/step
Epoch 5/100
12/12 - 0s - loss: 0.8264 - accuracy: 0.5014 - val_loss: 0.8081 - val_accuracy: 0.7500 - 58ms/epoch - 5ms/step
Epoch 6/100
12/12 - 0s - loss: 0.7941 - accuracy: 0.8319 - val_loss: 0.7817 - val_accuracy: 0.8000 - 52ms/epoch - 4ms/step
Epoch 7/100
12/12 - 0s - loss: 0.7683 - accuracy: 0.7647 - val_loss: 0.7587 - val_accuracy: 0.8500 - 50ms/epoch - 4ms/step
Epoch 8/100
12/12 - 0s - loss: 0.7454 - accuracy: 0.8768 - val_loss: 0.7410 - val_accuracy: 0.9000 - 50ms/epoch - 4ms/step
Epoch 9/100
12/12 - 0s - l

(0.351694792509079, 0.875)